<a href="https://colab.research.google.com/github/sejeong-park/2020-AI/blob/master/%ED%95%B4%EB%B9%99%20%EC%A0%95%EB%8F%84%20%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Would remove:
    /usr/local/bin/kaggle
    /usr/local/lib/python3.6/dist-packages/kaggle-1.5.6.dist-info/*
    /usr/local/lib/python3.6/dist-packages/kaggle/*
Proceed (y/n)? ㅛ
Your response ('ㅛ') was not one of the expected responses: y, n
Proceed (y/n)? y
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 6.9MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 3.5 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=e7bbbe1436e9dfabc5494534e265db77a0f3fa0cf3787fa0c34c0e8ead8a8830
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle
!ls -lha kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

-rw-r--r-- 1 root root 67 Jun 15 08:51 kaggle.json


In [ ]:
! kaggle competitions download -c prediction-of-sea-ice

  0% 0.00/12.8k [00:00<?, ?B/s]
100% 12.8k/12.8k [00:00<00:00, 9.62MB/s]


In [ ]:
!unzip prediction-of-sea-ice.zip

Archive:  prediction-of-sea-ice.zip
  inflating: seaice_sample.csv       
  inflating: seaice_test.csv         
  inflating: seaice_train.csv        


In [ ]:
from sklearn.preprocessing import RobustScaler
import pandas as pd
import torch
import torch.optim as optim
import numpy as np
import random
torch.manual_seed(1)

In [ ]:
seaice_data = pd.read_csv('seaice_train.csv')

In [ ]:
seaice_data

,year,month,car_reg,cow,gdp_,Carbon,seaice_extent
0,1978,11,47.000,1.211070e+14,7.214118e+06,333.99,11.645133
1,1978,12,48.000,1.211070e+14,7.618305e+06,335.26,13.605867
2,1979,1,49.000,1.211070e+09,8.022491e+06,336.22,15.414000
3,1979,2,49.275,1.211565e+09,8.092499e+06,336.96,16.175286
4,1979,3,49.550,1.212061e+09,8.162507e+06,337.80,16.341938
...,...,...,...,...,...,...,...
470,2019,1,2368.000,1.496833e+09,6.293426e+07,NaN,13.555677
471,2019,2,2370.300,1.496643e+09,6.307988e+07,NaN,14.398357
472,2019,3,2372.600,1.496452e+09,6.322549e+07,NaN,14.551968
473,2019,4,2374.900,1.496262e+09,6.337111e+07,NaN,13.454600


In [ ]:
from datetime import datetime as dt
from dateutil.parser import parse
import time

In [ ]:
def toYearFraction(date):
    def sinceEpoch(date): # returns seconds since epoch
        return time.mktime(date.timetuple())
    s = sinceEpoch

    year = date.year
    startOfThisYear = dt(year=year, month=1, day=1)
    startOfNextYear = dt(year=year+1, month=1, day=1)
    yearElapsed = s(date) - s(startOfThisYear)
    yearDuration = s(startOfNextYear) - s(startOfThisYear)
    # 초단위로 1년 / 현재날짜 의 비율 
    fraction = yearElapsed/yearDuration
    return date.year + fraction

In [ ]:
train_decimal_year = []

train_decimal_year.append(toYearFraction(parse("1978-11-15")))
train_decimal_year.append(toYearFraction(parse("1978-12-15")))

for i in range(1979, 2019):
  if i == 2016:
    continue
  for m in range(1 , 13):
    flag = int(m / 10)
    if flag == 0:
      date = ("{}-0{}-15".format(i,m))
    else:
      date = ("{}-{}-15".format(i,m))
    train_decimal_year.append(toYearFraction(parse(date)))

train_decimal_year.append(toYearFraction(parse("2019-01-15")))
train_decimal_year.append(toYearFraction(parse("2019-02-15")))
train_decimal_year.append(toYearFraction(parse("2019-03-15")))
train_decimal_year.append(toYearFraction(parse("2019-04-15")))
train_decimal_year.append(toYearFraction(parse("2019-05-15")))

In [ ]:
train_decimal_year = np.array(train_decimal_year)
train_decimal_year.shape

(475,)

In [ ]:
seaice_data = seaice_data.drop(['month'],axis = 1)

In [ ]:
for i in range(475):
  seaice_data.iloc[i,0] = train_decimal_year[i]

In [ ]:
seaice_data.tail(30)
seaice_data.iloc[447:475]

,year,car_reg,cow,gdp_,Carbon,seaice_extent
447,2017.123288,2258.61667,1.474659e+09,58654437.47,NaN,14.119429
448,2017.200000,2264.23333,1.476030e+09,58854796.29,NaN,14.290065
449,2017.284932,2269.85000,1.477401e+09,59055155.10,NaN,13.753467
450,2017.367123,2275.46667,1.478773e+09,59255513.91,NaN,12.630452
451,2017.452055,2281.08333,1.480144e+09,59455872.72,NaN,10.756400
452,2017.534247,2286.70000,1.481516e+09,59656231.54,NaN,7.939355
453,2017.619178,2292.31667,1.482887e+09,59856590.35,NaN,5.480645
454,2017.704110,2297.93333,1.484259e+09,60056949.16,NaN,4.821900
455,2017.786301,2303.55000,1.485630e+09,60257307.97,NaN,6.767129
456,2017.871233,2309.16667,1.487002e+09,60457666.79,NaN,9.492833


In [ ]:
x_carbon = np.array(seaice_data.iloc[0:447,0:4])
x_seaice = np.array(seaice_data.iloc[0:447,5])
y_carbon = np.array(seaice_data.iloc[0:447,4])

In [ ]:
x_carbon

array([[1.97887123e+03, 4.70000000e+01, 1.21107000e+14, 7.21411850e+06],
       [1.97895342e+03, 4.80000000e+01, 1.21107000e+14, 7.61830475e+06],
       [1.97903836e+03, 4.90000000e+01, 1.21106969e+09, 8.02249138e+06],
       ...,
       [2.01587123e+03, 2.16675000e+03, 1.46518016e+09, 5.55059597e+07],
       [2.01595342e+03, 2.17352500e+03, 1.46678969e+09, 5.56851165e+07],
       [2.01703836e+03, 2.25300000e+03, 1.47328713e+09, 5.84540787e+07]])

In [ ]:
scaler = RobustScaler()
x_carbon_s = scaler.fit_transform(x_carbon)

In [ ]:
x_carbon_s

array([[-9.99902314e-01, -7.03654222e-01,  9.73414858e+05,
        -7.36492798e-01],
       [-9.95479273e-01, -7.02947978e-01,  9.73414858e+05,
        -7.20516815e-01],
       [-9.90908796e-01, -7.02241734e-01, -8.77534553e-01,
        -7.04540817e-01],
       ...,
       [ 9.91203666e-01,  7.93406040e-01,  1.16493702e+00,
         1.17230452e+00],
       [ 9.95626707e-01,  7.98190842e-01,  1.17787400e+00,
         1.17938592e+00],
       [ 1.05401086e+00,  8.54319567e-01,  1.23009863e+00,
         1.28883273e+00]])

In [ ]:
x_ctrain = torch.FloatTensor(x_carbon_s)
y_ctrain = torch.FloatTensor(np.transpose(y_carbon[np.newaxis]))

In [ ]:
x_ctrain

tensor([[-9.9990e-01, -7.0365e-01,  9.7341e+05, -7.3649e-01],
        [-9.9548e-01, -7.0295e-01,  9.7341e+05, -7.2052e-01],
        [-9.9091e-01, -7.0224e-01, -8.7753e-01, -7.0454e-01],
        ...,
        [ 9.9120e-01,  7.9341e-01,  1.1649e+00,  1.1723e+00],
        [ 9.9563e-01,  7.9819e-01,  1.1779e+00,  1.1794e+00],
        [ 1.0540e+00,  8.5432e-01,  1.2301e+00,  1.2888e+00]])

In [ ]:
y_ctrain

tensor([[333.9900],
        [335.2600],
        [336.2200],
        [336.9600],
        [337.8000],
        [339.0400],
        [339.6500],
        [339.0500],
        [337.6400],
        [335.6900],
        [334.1100],
        [334.1700],
        [335.5200],
        [336.8600],
        [337.8800],
        [338.6800],
        [339.6000],
        [340.8700],
        [341.4800],
        [340.8700],
        [339.4500],
        [337.4900],
        [335.9000],
        [335.9300],
        [337.2400],
        [338.5200],
        [339.4700],
        [340.2100],
        [341.0400],
        [342.2400],
        [342.8200],
        [342.1500],
        [340.6900],
        [338.6700],
        [337.0300],
        [337.0400],
        [338.3600],
        [339.6600],
        [340.6300],
        [341.3700],
        [342.2000],
        [343.4100],
        [343.9900],
        [343.3100],
        [341.8300],
        [339.8000],
        [338.1400],
        [338.1400],
        [339.4600],
        [340.7800],


In [ ]:
W = torch.zeros((4,1),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

In [ ]:
import torch.nn.functional as F
optimizer = optim.Adam([W,b], lr = 0.1)
loss = torch.nn.MSELoss()

nb_epochs = 100000
for epochs in range(nb_epochs + 1):
  h = x_ctrain.matmul(W)+ b
  cost = loss(h,y_ctrain)
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epochs % 1000 == 0:
    print("epochs {}/{:4d}, cost {:.6f}".format(epochs, nb_epochs, cost.item()))

epochs 0/100000, cost 134440.484375
epochs 1000/100000, cost 70788.914062
epochs 2000/100000, cost 31711.923828
epochs 3000/100000, cost 11228.016602
epochs 4000/100000, cost 2865.727295
epochs 5000/100000, cost 655.416138
epochs 6000/100000, cost 362.652435
epochs 7000/100000, cost 322.211243
epochs 8000/100000, cost 87414.570312
epochs 9000/100000, cost 267.119904
epochs 10000/100000, cost 240.671463
epochs 11000/100000, cost 4737.510742
epochs 12000/100000, cost 197.654449
epochs 13000/100000, cost 179.114777
epochs 14000/100000, cost 107875.375000
epochs 15000/100000, cost 148.353561
epochs 16000/100000, cost 134.022751
epochs 17000/100000, cost 121.649582
epochs 18000/100000, cost 112.311974
epochs 19000/100000, cost 100.485497
epochs 20000/100000, cost 91.316338
epochs 21000/100000, cost 1435.713623
epochs 22000/100000, cost 75.653687
epochs 23000/100000, cost 68.967987
epochs 24000/100000, cost 62.888599
epochs 25000/100000, cost 57.391586
epochs 26000/100000, cost 25020.695312


In [ ]:
carbon_x_test = np.array(seaice_data.iloc[447:475,0:4])
carbon_xs_test = scaler.fit_transform(carbon_x_test)
carbon_xs_test

array([[-0.99512789, -1.18629724, -1.46670285, -1.07535441],
       [-0.92691839, -1.09281932, -1.34525428, -0.99363119],
       [-0.85140073, -0.99934123, -1.22380572, -0.91190797],
       [-0.77831912, -0.90586314, -1.10235715, -0.83018475],
       [-0.70280146, -0.81238522, -0.98090858, -0.74846154],
       [-0.62971985, -0.71890713, -0.85945993, -0.66673831],
       [-0.55420219, -0.62542904, -0.73801136, -0.5850151 ],
       [-0.47868453, -0.53195112, -0.6165628 , -0.50329188],
       [-0.40560292, -0.43847303, -0.49511423, -0.42156866],
       [-0.33008526, -0.34499494, -0.37366567, -0.33984544],
       [-0.25700365, -0.25151702, -0.2522171 , -0.25812222],
       [-0.18148599, -0.16469612, -0.13076854, -0.17639901],
       [-0.10596833, -0.09881774, -0.07846114, -0.1058394 ],
       [-0.03775883, -0.03293919, -0.02615374, -0.0352798 ],
       [ 0.03775883,  0.03293919,  0.02615374,  0.0352798 ],
       [ 0.11084044,  0.09881757,  0.07846114,  0.1058394 ],
       [ 0.1863581 ,  0.

In [ ]:
x_ctest = torch.FloatTensor(carbon_xs_test)
carbon_pred = x_ctest.matmul(W)+b
carbon_pred = carbon_pred.detach().numpy()

In [ ]:
carbon_pred

array([[329.22928],
       [331.75714],
       [334.37436],
       [336.9618 ],
       [339.579  ],
       [342.16644],
       [344.78366],
       [347.40088],
       [349.9883 ],
       [352.60553],
       [355.19296],
       [357.83054],
       [360.26172],
       [362.60352],
       [365.03473],
       [367.4361 ],
       [369.86728],
       [372.26868],
       [374.69986],
       [377.13104],
       [379.53244],
       [381.96362],
       [384.36502],
       [386.77078],
       [389.01593],
       [391.1717 ],
       [393.41684],
       [395.6322 ]], dtype=float32)

In [ ]:
index = 0
for i in range(447,475):
   seaice_data.iloc[i,4] = carbon_pred[index]
   index = index + 1

In [ ]:
x_data = np.array(seaice_data.iloc[:,0:5])
xs_data = scaler.fit_transform(x_data)
y_data = np.transpose(np.array(seaice_data.iloc[:,5])[np.newaxis])

In [ ]:
x_train = torch.FloatTensor(xs_data)
y_train = torch.FloatTensor(y_data)

In [ ]:
x_train

tensor([[-9.9997e-01, -7.1508e-01,  9.7430e+05, -7.5721e-01, -9.3770e-01],
        [-9.9580e-01, -7.1439e-01,  9.7430e+05, -7.4180e-01, -8.9806e-01],
        [-9.9150e-01, -7.1370e-01, -8.9708e-01, -7.2640e-01, -8.6809e-01],
        ...,
        [ 1.0421e+00,  8.8320e-01,  1.3988e+00,  1.3773e+00,  8.4723e-01],
        [ 1.0464e+00,  8.8478e-01,  1.3973e+00,  1.3829e+00,  9.1731e-01],
        [ 1.0506e+00,  8.8636e-01,  1.3958e+00,  1.3884e+00,  9.8646e-01]])

In [ ]:
y_train

tensor([[11.6451],
        [13.6059],
        [15.4140],
        [16.1753],
        [16.3419],
        [15.4468],
        [13.8569],
        [12.5300],
        [10.3112],
        [ 8.0411],
        [ 7.0511],
        [ 8.7479],
        [10.9431],
        [13.2823],
        [14.8619],
        [15.9551],
        [16.0413],
        [15.4291],
        [13.7926],
        [12.2046],
        [10.1001],
        [ 7.9843],
        [ 7.6671],
        [ 9.1827],
        [11.3829],
        [13.5572],
        [14.9097],
        [15.6041],
        [15.6315],
        [15.0099],
        [13.8016],
        [12.4297],
        [10.2710],
        [ 7.8443],
        [ 7.1385],
        [ 8.8563],
        [10.9290],
        [13.2823],
        [15.1767],
        [15.9735],
        [16.0439],
        [15.4660],
        [13.9732],
        [12.4761],
        [10.3671],
        [ 8.1395],
        [ 7.3017],
        [ 9.4211],
        [11.6275],
        [13.6049],
        [14.9424],
        [16.0061],
        [16.

In [ ]:
x_W = torch.zeros((5,1), requires_grad=True)
x_b = torch.zeros(1, requires_grad=True)

In [ ]:
optimizer_1 = optim.Adam([x_W,x_b], lr = 1e-3)
loss = torch.nn.MSELoss()

nb_epochs = 50000
for epochs in range(nb_epochs + 1):
  h_1 = x_train.matmul(x_W)+ x_b

  cost_1 = loss(h_1,y_train)
  optimizer_1.zero_grad()
  cost_1.backward()
  optimizer_1.step()

  if epochs % 1000 == 0:
    print("epochs {}/{:4d}, cost {:.6f}".format(epochs, nb_epochs, cost_1.item()))

epochs 0/50000, cost 142.927872
epochs 1000/50000, cost 117.434166
epochs 2000/50000, cost 95.692879
epochs 3000/50000, cost 77.065063
epochs 4000/50000, cost 61.239849
epochs 5000/50000, cost 47.958187
epochs 6000/50000, cost 60.015392
epochs 7000/50000, cost 28.298645
epochs 8000/50000, cost 21.581720
epochs 9000/50000, cost 16.684219
epochs 10000/50000, cost 13.415312
epochs 11000/50000, cost 11.509745
epochs 12000/50000, cost 10.592340
epochs 13000/50000, cost 15.059517
epochs 14000/50000, cost 10.168923
epochs 15000/50000, cost 10.115709
epochs 16000/50000, cost 10.070675
epochs 17000/50000, cost 10.028770
epochs 18000/50000, cost 9.992998
epochs 19000/50000, cost 9.957221
epochs 20000/50000, cost 25.196218
epochs 21000/50000, cost 9.898582
epochs 22000/50000, cost 11.381709
epochs 23000/50000, cost 9.850769
epochs 24000/50000, cost 17.874884
epochs 25000/50000, cost 9.811983
epochs 26000/50000, cost 10.416400
epochs 27000/50000, cost 9.780699
epochs 28000/50000, cost 9.767356
epo

In [ ]:
test_data = pd.read_csv('seaice_test.csv')
test_data

,year,month,car_reg,cow,gdp_,Carbon
0,2016,1,2180.00000,1468399225,55864273.29,402.82
1,2016,2,2186.04167,1468806550,56080090.40,403.84
2,2016,3,2192.08333,1469213875,56295907.52,404.98
3,2016,4,2198.12500,1469621201,56511724.63,406.51
4,2016,5,2204.16667,1470028526,56727541.75,407.28
5,2016,6,2210.20833,1470435851,56943358.86,406.66
6,2016,7,2216.25000,1470843176,57159175.98,405.14
7,2016,8,2222.29167,1471250501,57374993.09,403.02
8,2016,9,2228.33333,1471657826,57590810.20,401.33
9,2016,10,2234.37500,1472065152,57806627.32,401.48


In [ ]:
test_decimal_year = []

for m in range(1 , 13):
  flag = int(m / 10)
  if flag == 0:
    date = ("2016-0{}-15".format(m))
  else:
    date = ("2016-{}-15".format(m))
  test_decimal_year.append(toYearFraction(parse(date)))

test_decimal_year = np.array(test_decimal_year )
test_decimal_year

array([2016.03825137, 2016.12295082, 2016.20218579, 2016.28688525,
       2016.36885246, 2016.45355191, 2016.53551913, 2016.62021858,
       2016.70491803, 2016.78688525, 2016.8715847 , 2016.95355191])

In [ ]:
test_data = test_data.drop(['month'],axis = 1)

for i in range(12):
  test_data.iloc[i,0] = test_decimal_year[i]

In [ ]:
tests_data = scaler.fit_transform(np.array(test_data))

tests_data = torch.FloatTensor(tests_data)
print(tests_data)

tensor([[-0.9926, -1.0000, -1.0000, -1.0000, -0.5692],
        [-0.8083, -0.8182, -0.8182, -0.8182, -0.1632],
        [-0.6360, -0.6364, -0.6364, -0.6364,  0.2905],
        [-0.4517, -0.4545, -0.4545, -0.4545,  0.8995],
        [-0.2734, -0.2727, -0.2727, -0.2727,  1.2060],
        [-0.0892, -0.0909, -0.0909, -0.0909,  0.9592],
        [ 0.0892,  0.0909,  0.0909,  0.0909,  0.3542],
        [ 0.2734,  0.2727,  0.2727,  0.2727, -0.4896],
        [ 0.4577,  0.4545,  0.4545,  0.4545, -1.1622],
        [ 0.6360,  0.6364,  0.6364,  0.6364, -1.1025],
        [ 0.8202,  0.8182,  0.8182,  0.8182, -0.4657],
        [ 0.9985,  1.0000,  1.0000,  1.0000,  0.1632]])


In [ ]:
predict_h = tests_data.matmul(x_W) + x_b
predict_h

tensor([[12.6326],
        [12.8007],
        [13.1023],
        [13.5830],
        [13.6278],
        [12.7907],
        [11.4319],
        [ 9.6753],
        [ 8.1824],
        [ 7.8472],
        [ 8.3707],
        [ 8.9120]], grad_fn=<AddBackward0>)

In [ ]:
submit=pd.read_csv('seaice_sample.csv')
submit

,month,seaice_extent
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN
5,6,NaN
6,7,NaN
7,8,NaN
8,9,NaN
9,10,NaN


In [ ]:
for i in range(len(predict_h)):
  submit['seaice_extent'][i]=predict_h[i].item()

submit

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,month,seaice_extent
0,1,12.632626
1,2,12.800704
2,3,13.102301
3,4,13.582975
4,5,13.627802
5,6,12.790665
6,7,11.431870
7,8,9.675328
8,9,8.182350
9,10,7.847157


In [ ]:
submit.to_csv('submit.csv', index=False, header=True)

In [ ]:
!kaggle competitions submit -c prediction-of-sea-ice -f submit.csv -m "17011885"

100% 270/270 [00:03<00:00, 70.1B/s]
Successfully submitted to Prediction Of Sea Ice